## merge IBM and Internet geographical data

-----

The code in this notebook merges IBM internal network data and Internet geography files for use in IBM Streaming Analytics, such as the NetflowViewer demonstration and cyber-security applications:

* Data for the IBM internal network comes from an Excel file provided by Mark Harvey (Mark_Harvey@uk.ibm.com). The file contains separate spreadsheets for IBM marketing regions US, EMEA, and AP. These spreadsheets list the IP subnet addresses IBM has assigned to its offices in each region, along with country, city, and street address (but not state/province/territory or latitude/longitude). All of these subnets are within the class A subnet 9.xxx.xxx.xxx assigned to IBM.

* The Internet geography data comes from CSV files provided by [MaxMind, Inc.](https://www.maxmind.com/en/home) as [GeoLite2 data](https://dev.maxmind.com/geoip/geoip2/geolite2/). This notebook downloads the 'GeoLite2' data, which MaxMind offers free of charge. The files list IP subnets in the Internet, along with country, state/province/territory, city, latitude, and longitude. MaxMind updates this data once a month, and this notebook downloads the data into a directory whose name includes the date of the files.

This notebook geocodes the IBM data with state/province/territory and latitude/longitude data from Google using its Geocoding API service. Google limits this service to 2,500 requests per day, which is sufficient for several runs of the cell below that geocodes a list of IBM locations. 

Finally, this notebook merges the geocoded IBM data into the MaxMind CSV files. It also generates a separate CSV file containing a [geohash code](https://en.wikipedia.org/wiki/Geohash) for each city's latitude/longitude. All of the resulting CSV files are packed into a ZIP file for transfer to Streaming Analytics projects:

* [GeoLite2-City-Blocks-IPv4.csv](merged/GeoLite2-City-Blocks-IPv4.csv)
* [GeoLite2-City-Blocks-IPv6.csv](merged/GeoLite2-City-Blocks-IPv6.csv)
* [GeoLite2-City-Locations-en.csv](merged/GeoLite2-City-Locations-en.csv)
* [GeoLite2-City-Geohashes-en.csv](merged/GeoLite2-City-Geohashes-en.csv)

To use this notebook, you will need to provide these things:

* an Excel file named 'report_IGA_Global_Q1_2016.xlsx' containing IBM internal network data

* a Google Maps geocoding API key for a valid Google account

There are detailed instructions these steps in the cells below.

-----
Run this cell once to install additional function packages:

In [44]:
!pip install --user googlemaps
!pip install --user geohash2

-----
Run this cell to set up the notebook's runtime environment:

In [45]:
import os
import math
import pprint
import shutil
import zipfile
import types

# load functions for maniulating matrixes 
import pandas as pd
pd.set_option('max_rows', 15)

# load functions for reading and writing byte streams
from io import BytesIO

# load functions for reading URLs
from urllib.request import urlopen

# load functions for reading and writing Cloud Object Storage
import ibm_boto3
from ibm_botocore.client import Config

# load functions for the Google Maps geocoding API
import googlemaps

# load functions for converting latitude/longitude coordinates into geohash codes
import geohash2

# create a local directory for staging merged CSV files
os.makedirs('merged', exist_ok=True)

-----
Then, provide the Excel file containing IBM internal network data available to this notebook. To do this, copy the file to the notebook's Cloud Object Storage bucket and create an HTTP client for reading and writing files:

* open the 'Files' panel by clicking the 'Data' icon in the upper-right corner of this DSX project,

* drag Excel file 'report_IGA_Global_Q1_2016.xlsx' from your laptip to the 'drop' area in the 'Files' panel,

* position the cursor at the top of the next cell and click 'Insert to code -> Insert Credentials' in the 'Files' panel,

* make sure the name of the inserted variable is 'credentials', and

* run the cell to set the credentials in the variable.

In [46]:
# The code was removed by DSX for sharing.

-----
Run the following cell to read IBM network addresses and locations from the Excel file's spreadsheets into 'pandas' frames:

In [47]:
print('running ...')

# create a Cloud Object Store HTTP client with the bucket's credentials
cosClient = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

# get a byte stream for reading the Excel file from the bucket
excelStream = cosClient.get_object(Bucket=credentials['BUCKET'], Key=credentials['FILE'])['Body']

# add an iterator method to the stream object so pandas will accept it as a file-like object
def __iter__(self): return 0
if not hasattr(excelStream, "__iter__"): excelStream.__iter__ = types.MethodType(__iter__, excelStream) 

# prepare to read the byte stream from Cloud Object Store as an Excel file
excelFile = pd.ExcelFile(excelStream)

# concatenate the data in each spreadsheet of the Excel file into a single pandas DataFrame
ibmData = pd.concat( map(lambda sheet: pd.read_excel(excelFile, sheet, header=0), excelFile.sheet_names) )

# correct some misencoded city names in the 'report_IGA_Global_Q1_2016.xlsx' spreadsheets
ibmCorrections = { 'S?O Paulo': 'Sao Paulo', 'Quer?Taro': 'Queretaro' }
for name in ibmCorrections:
    ibmData.loc[ibmData['# City']==name, '# City'] = ibmCorrections[name]

# create a frame with the country, city, and street address of each IBM internal IP subnet
    ibmNetworks = ibmData[['# Network Container', '# Country', '# City', '# Street']].dropna()
ibmNetworks = ibmNetworks[ibmNetworks['# Network Container'].str.match("[0-9./]+")]
ibmNetworks = ibmNetworks.drop_duplicates('# Network Container',keep='first')
ibmNetworks.columns = ['network', 'country_name', 'city_name', 'street_address']    
                       
# create a frame with the country, city, and street address of each IBM location, and empty columns for more geographical data
ibmLocations = ibmNetworks.drop('network',axis=1).drop_duplicates(['country_name', 'city_name', 'street_address'])
ibmLocations['geoname_id'] = range(len(ibmLocations))
ibmLocations['country_iso_code'] = None
ibmLocations['subdivision_1_iso_code'] = None
ibmLocations['subdivision_1_name'] = None
ibmLocations['subdivision_2_iso_code'] = None
ibmLocations['subdivision_2_name'] = None
ibmLocations['postal_code'] = None
ibmLocations['latitude'] = None
ibmLocations['longitude'] = None

print('... done')

running ...
... done


-----
Run the next cell to download and unpack Internet network and location data from MaxMind into 'pandas' frames:

In [48]:
maxmindURL = 'http://geolite.maxmind.com/download/geoip/database/GeoLite2-City-CSV.zip'

print('running ...')

with urlopen(maxmindURL) as response:
    with zipfile.ZipFile(BytesIO(response.read())) as file:
        file.extractall()

maxmindDirectory = sorted( [ f for f in os.listdir() if os.path.isdir(f) and f.startswith('GeoLite2-City-CSV') ] )[-1]
maxmindLocations = pd.read_csv(maxmindDirectory + '/GeoLite2-City-Locations-en.csv', header=0)
maxmindNetworks = pd.read_csv(maxmindDirectory + '/GeoLite2-City-Blocks-IPv4.csv', header=0)

print('... done')

running ...
... done


-----
Next, get a Google geocoding API key for a valid Google account:

* In a browser, go to [Google](https://www.google.com/) and sign into an existing account or create a new account.

* Go to the [Google Geocoding Service](https://developers.google.com/maps/documentation/javascript/geocoding) page and follow the instructions to create a project and enable the geocoding API.

* Go to [Google Geocoding Service 'Get API Key'](https://developers.google.com/maps/documentation/geocoding/get-api-key), click on 'Get a Key', and then click the 'copy' button.

* paste the copied key into the next cell as the value of the 'googlemapsKey' variable 

* run the next cell to set key in the variable

Note that this service is limited to 2,500 requests per day. That is sufficient for several runs of the cell below that geocodes a list of IBM cities. 

In [49]:
# The code was removed by DSX for sharing.

Run the next cell to fill the empty geography columns for IBM locations with data from the Google geocoding service. Note that Google limits geocoding API requests to 2,500 per day. There are about 450 IBM locations, so you can run this cell several times in the same day before reaching the limit. After the limit is reached, the Google 'geocode()' function will return an error code for the remainder of the day.

In [50]:
def convertAddressToGeocode(client, address):
    result = client.geocode(address)
    ###########print('>>>>>>>>>>>client.geocode(' + address + ') returned:')
    #######pprint.pprint(result,width=150)
    if result is None: return None
    if len(result)<1: return None
    if 'address_components' not in result[0]: return None
    geocode = dict( [ (i['types'][0],{'long_name':i['long_name'],'short_name': i['short_name']}) for i in result[0]['address_components'] ] )
    geocode['latitude'] = result[0]['geometry']['location']['lat']
    geocode['longitude'] = result[0]['geometry']['location']['lng']
    ########print('>>>>>>>>>>>convertAddressToGeocode(' + address + ') returned:')
    ##########pprint.pprint(geocode,width=150)
    return geocode

K = 500

def geocodeIBMLocationRow(googlemapsClient, row):
    if row['geoname_id']>K: return row
    address = "IBM, " + row['street_address'] + ', ' + row['city_name'] + ', ' + row['country_name']
    geocode = convertAddressToGeocode(googlemapsClient, address)
    if geocode is None:
        address = "IBM, " + row['city_name'] + ', ' + row['country_name']
        geocode = convertAddressToGeocode(googlemapsClient, address)
    if geocode is None: 
        print('address not found: ' + address)
        return row
    try: 
        if 'country' in geocode: 
            row['country_iso_code'] = geocode['country']['short_name']
        if 'administrative_area_level_1' in geocode: 
            row['subdivision_1_iso_code'] = geocode['administrative_area_level_1']['short_name']
            row['subdivision_1_name'] = geocode['administrative_area_level_1']['long_name']
        if 'administrative_area_level_2' in geocode: 
            row['subdivision_2_iso_code'] = geocode['administrative_area_level_2']['short_name']
            row['subdivision_2_name'] = geocode['administrative_area_level_2']['long_name']
        if 'postal_code' in geocode: 
            row['postal_code'] = geocode['postal_code']['long_name']
        if 'latitude' in geocode: 
            row['latitude'] = geocode['latitude']
            row['longitude'] = geocode['longitude']
    except KeyError as e: 
        print(str(e) + 'not found for address ' + address) 
    return row

print('running ...')

# create an HTTP client for using the Google Maps geocoding API
googlemapsClient = googlemaps.Client(key=googlemapsKey)

# geocode each IBM location
ibmLocations = ibmLocations.apply(lambda row: geocodeIBMLocationRow(googlemapsClient, row), axis=1)

print('... done')

running ...
address not found: IBM, Acheson, Canada
address not found: IBM, Florenceville, Canada


KeyboardInterrupt: 

-----
Run the next cell to merge the MaxMind and IBM locations. The merged data will be written into a CSV file in the 'merged' directory named 'GeoLite2-City-Locations-en.csv'.

In [ ]:
print('running ...')

# create a frame of IBM locations with 'IBM' in the city name
ibmLocationsWithRelabeledCity = ibmLocations
ibmLocationsWithRelabeledCity['city_name'] = 'IBM ' + ibmLocations['city_name']

# merge the MaxMind and IBM location frames and store the result as a CSV file
mergedLocations = pd.concat([maxmindLocations,ibmLocationsWithRelabeledCity[ list( set(maxmindLocations.columns) & set(ibmLocationsWithRelabeledCity.columns) ) ]])
mergedLocations.to_csv('merged/GeoLite2-City-Locations-en.csv', index=False, float_format='%.9g', columns=maxmindLocations.columns)

print('... done')

-----
Run the next cell to merge the MaxMind and IBM networks. The merged data will be written into a CSV file in the 'merged' directory named 'GeoLite2-City-Blocks-IPv4.csv'.

In [ ]:
print('running ...')

# create a frame of IBM networks with 'IBM' in the city name
ibmNetworksWithRelabeledCity = ibmNetworks
ibmNetworksWithRelabeledCity['city_name'] = 'IBM ' + ibmNetworksWithRelabeledCity['city_name']

# create a frame of IBM locations indexed by country, city, and street address
ibmLocationsIndexed = ibmLocationsWithRelabeledCity.set_index(['country_name','city_name','street_address'])

# add country, city, and street address for each network in the IBM networks frame
ibmNetworksWithLocation = ibmNetworksWithRelabeledCity.join(ibmLocationsIndexed, on=['country_name','city_name','street_address'])

# merge the MaxMind and IBM network frames and store the result in a CSV file
mergedNetworks = pd.concat([maxmindNetworks,ibmNetworksWithLocation[ list( set(maxmindNetworks.columns) & set(ibmNetworksWithLocation.columns) ) ]])
mergedNetworks.to_csv('merged/GeoLite2-City-Blocks-IPv4.csv', index=False, float_format='%.9g', columns=maxmindNetworks.columns)
                                                        
print('... done')

-----
Run the next cell to calculate [geohash codes](https://en.wikipedia.org/wiki/Geohash) for the latitude/longitude coordinates of merged MaxMind and IBM locations. The geohashes, coordinates, and location data will be written into a CSV file in the 'merged' directory named 'GeoLite2-City-Geohashes-en.csv'.

In [ ]:
print('running ...')

# create a frame of locations indexed by ID number
mergedLocationsIndexed = mergedLocations.set_index('geoname_id')

# create a frame of geographical coordinates, that is, ID number, latitude, and longitude
mergedCoordinates = mergedNetworks[['geoname_id','latitude','longitude']].drop_duplicates()

# merge location and coordinate data and calculate geohash for each location's coordinates
mergedGeohashes = mergedCoordinates.join(mergedLocationsIndexed, on='geoname_id')
mergedGeohashes['geohash'] = mergedGeohashes.apply(lambda row: geohash2.encode(row['latitude'],row['longitude'],precision=6),axis=1)

# store the result in a CSV file
columns = ['geohash','latitude','longitude','geoname_id','country_iso_code','country_name','subdivision_1_iso_code','subdivision_1_name','subdivision_2_iso_code','subdivision_2_name','city_name']
mergedGeohashes.to_csv('merged/GeoLite2-City-Geohashes-en.csv', index=False, float_format='%.9g', columns=columns)
                                                        
print('... done')

-----
Finally, pack all of the merged CSV files into a ZIP package and copy it to the notebook's bucket in Cloud Object Storage ....

In [ ]:
resultPackage = 'mergedIBMandInternetGeographyData.zip'

print('running ...')

# add the MaxMind IPv6 network file to the ZIP package
shutil.copy(maxmindDirectory + '/GeoLite2-City-Blocks-IPv6.csv', 'merged')

# pack all result files into a ZIP package
with zipfile.ZipFile(resultPackage, 'w', compression=zipfile.ZIP_DEFLATED) as zipFile:
    for file in os.listdir('merged'):
        zipFile.write('merged/'+file, file)

# write the ZIP file to the notebook's bucket in Cloud Object Storage
cosClient.upload_file(Filename=resultPackage, Bucket=credentials['BUCKET'], Key=resultPackage)

print('... done')

-----
To download the ZIP package containing the results of merging IBM and Internet geography data, do this:

* In a browser, go to this notebook's project page

* open the 'Files' panel by clicking the 'Find and Add Data' icon in the upper-right corner of the project page,

* check the box next to 'mergedIBMandInternetGeographyData.zip'

* select 'Download' from the pop-up menu in the 'Files' panel

\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/

-----
Optionally, run this last cell to clean up the notebook's runtime environment. This is really not necessary.

In [ ]:
#!rm -rf *
#!pip uninstall -y googlemaps geohash2 

In [51]:
ls -al

total 0
drwx------  4 sa73-1acf9232f65bd2-cf1c60ef4a00 users 4096 Feb 11 14:46 ./
drwx------ 11 sa73-1acf9232f65bd2-cf1c60ef4a00 users 4096 Feb 11 11:48 ../
drwx------  2 sa73-1acf9232f65bd2-cf1c60ef4a00 users 4096 Feb 11 14:46 GeoLite2-City-CSV_20180206/
drwx------  2 sa73-1acf9232f65bd2-cf1c60ef4a00 users 4096 Feb 11 14:46 merged/
